<a href="https://colab.research.google.com/github/preethi-raddi/feature-engineering-ASAG/blob/main/paper1_code1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load necessary libraries
install.packages(c("caret", "e1071", "randomForest", "tm","text2vec"))

library(caret)
library(e1071)
library(randomForest)
library(tm)
library(text2vec)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘timeDate’, ‘MatrixExtra’, ‘float’, ‘RhpcBLASctl’, ‘RcppArmadillo’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’, ‘proxy’, ‘NLP’, ‘slam’, ‘BH’, ‘rsparse’, ‘mlapi’, ‘lgr’


Loading required package: ggplot2

Loading required package: lattice

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate




In [3]:
data <- read.csv("/content/mohler_dataset_edited.csv")

head(data,5)

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,"For us it is usually a set, get, and displayMessge or other messages that tell you what properties a class has but does not reveal its implementations. You could have anything you want in your own class definition.",1,3,2
2,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,The attributes of said class. Also whether or not it is a subclass. Also whether it is public private or protected.,4,4,4
3,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,Information telling the compiler what data members and member functions belong to the class.,5,5,5
4,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,"Since class is user defined, the body of the declaration can contain members, that can be either data or function declarations, and optionally access specifiers. Example for specifiers would be public, private, and protected. There is the keyword Class and the Class body is enclosed with brackets ({}).",5,5,5
5,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,An object and data.,1,3,2


In [4]:
# Preprocessing Function for Text
preprocess_text <- function(text) {
  corpus <- Corpus(VectorSource(text))

  # Apply text preprocessing
  corpus <- tm_map(corpus, content_transformer(tolower))   # Convert to lowercase
  corpus <- tm_map(corpus, removePunctuation)             # Remove punctuation
  corpus <- tm_map(corpus, removeWords, stopwords("en"))  # Remove stopwords
  corpus <- tm_map(corpus, stripWhitespace)               # Remove extra whitespace

  # Handle empty documents: Replace with "emptydoc"
  for (i in seq_along(corpus)) {
    if (nchar(as.character(corpus[[i]])) == 0) {  # Use as.character(corpus[[i]])
      corpus[[i]] <- PlainTextDocument("emptydoc")  # assign new PlainTextDocument
    }
  }

  # Extract all processed text as a character vector
  cleaned_texts <- sapply(corpus, as.character)  # Use as.character(doc)

  return(cleaned_texts)
}

# Apply Preprocessing
data$StudentAnswer <- sapply(data$student_answer, preprocess_text)
data$ModelAnswer <- sapply(data$desired_answer, preprocess_text)

Streaming output truncated to the last 5000 lines.
Warning message in tm_map.SimpleCorpus(corpus, content_transformer(tolower)):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removePunctuation):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removeWords, stopwords("en")):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, stripWhitespace):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, content_transformer(tolower)):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removePunctuation):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removeWords, stopwords("en")):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, stripWhitespace):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, content_transformer(tolower)):
“transformation drops documents

In [5]:
# Feature Engineering (e.g., Cosine Similarity)
vectorizer <- vocab_vectorizer(vocabulary = create_vocabulary(itoken(data$StudentAnswer)))
dtm_student <- create_dtm(itoken(data$StudentAnswer), vectorizer)
dtm_model <- create_dtm(itoken(data$ModelAnswer), vectorizer)
dtm_model
dtm_student
prod <- dtm_model*dtm_student
dim(prod)
prod<-array(prod,dim = c(315,534))
dim(dtm_model)
dtm_model<-array(dtm_model,dim = c(315,534))
dtm_student<-array(dtm_student,dim = c(315,534))
cosine_similarity <- rowSums(prod) /
  (sqrt(rowSums(dtm_student^2)) * sqrt(rowSums(dtm_model^2)))

data$CosineSimilarity <- cosine_similarity

  [[ suppressing 33 column names ‘10’, ‘18’, ‘1dequeue’ ... ]]



315 x 534 sparse Matrix of class "dgCMatrix"
                                                                            
1   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
2   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
3   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
4   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
5   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
6   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
7   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
8   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
9   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
10  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
11  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
12  . . . . . . . . . . . . . .

  [[ suppressing 33 column names ‘10’, ‘18’, ‘1dequeue’ ... ]]



315 x 534 sparse Matrix of class "dgCMatrix"
                                                                            
1   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
2   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
3   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
4   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
5   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
6   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
7   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
8   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
9   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
10  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
11  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
12  . . . . . . . . . . . . . .

[1] 315 534

[1] 315 534

In [6]:
str(data)

'data.frame':	315 obs. of  10 variables:
 $ id              : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ question        : chr  "What is typically included in a class definition?" "What is typically included in a class definition?" "What is typically included in a class definition?" "What is typically included in a class definition?" ...
 $ desired_answer  : chr  "Data members (attributes) and member functions." "Data members (attributes) and member functions." "Data members (attributes) and member functions." "Data members (attributes) and member functions." ...
 $ student_answer  : chr  "For us it is usually a set, get, and displayMessge or other messages that tell you what properties a class has "| __truncated__ "The attributes of said class. Also whether or not it is a subclass. Also whether it is public private or protected." "Information telling the compiler what data members and member functions belong to the class." "Since class is user defined, the body of the declara

In [7]:
#to check if there are character columns
char_columns <- sapply(data, is.character)
print(char_columns)

              id         question   desired_answer   student_answer 
           FALSE             TRUE             TRUE             TRUE 
        score_me      score_other        score_avg    StudentAnswer 
           FALSE            FALSE            FALSE             TRUE 
     ModelAnswer CosineSimilarity 
            TRUE            FALSE 


In [8]:
#convert the char columns to factor columns
data[char_columns] <- lapply(data[char_columns], factor)
# to check the data
str(data)

'data.frame':	315 obs. of  10 variables:
 $ id              : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ question        : Factor w/ 11 levels "How are arrays passed to functions?",..: 11 11 11 11 11 11 11 11 11 11 ...
 $ desired_answer  : Factor w/ 10 levels "by reference.",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ student_answer  : Factor w/ 277 levels "- A function with access to a linked list's head pointer has access to the entire list.        - Pass the head "| __truncated__,..: 87 221 101 210 33 106 70 5 68 133 ...
 $ score_me        : num  1 4 5 5 1 2 5 5 5 3 ...
 $ score_other     : num  3 4 5 5 3 4 5 5 5 4 ...
 $ score_avg       : num  2 4 5 5 2 3 5 5 5 3.5 ...
 $ StudentAnswer   : Factor w/ 248 levels " access specifiers public private",..: 97 12 149 231 65 91 129 23 129 176 ...
 $ ModelAnswer     : Factor w/ 9 levels " iteration"," reference",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ CosineSimilarity: num  0 0.115 0.596 0.141 0.316 ...


In [9]:
is.na(data)

id,question,desired_answer,student_answer,score_me,score_other,score_avg,StudentAnswer,ModelAnswer,CosineSimilarity
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [10]:
# Remove rows with any missing values
data <- na.omit(data)

# Check levels of all factor columns
factor_levels <- sapply(data, function(x) if (is.factor(x)) levels(x))
print(factor_levels)

$id
NULL

$question
 [1] "How are arrays passed to functions?"                                                             
 [2] "How are linked lists passed as arguments to a function?"                                         
 [3] "How can you implement a queue with an array?"                                                    
 [4] "What are the access-specifiers that can be used in a C++ class definition?"                      
 [5] "What are the main operations associated with a stack?"                                           
 [6] "What are the two main functions defined by a stack?"                                             
 [7] "What is the alternative way to solve a problem that could be solved through recursive functions?"
 [8] "What is the difference between a data member and a local variable inside a member function?"     
 [9] "What is the scope of global variables?"                                                          
[10] "What is the stack operation correspond

In [14]:
install.packages(c("caret","kernlab"))
library(kernlab)
library(caret)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘kernlab’


The following object is masked from ‘package:ggplot2’:

    alpha




In [15]:
# Define Regression Models for Ensemble
models <- list(
  lm = train(CosineSimilarity ~ ., data = data, method = "lm"),  # Linear Regression
  rf = train(CosineSimilarity ~ ., data = data, method = "rf"),  # Random Forest
  svm = train(CosineSimilarity ~ ., data = data, method = "svmRadial")  # SVM Regression
)


Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(modelFit, newdata):
“prediction from rank-deficient fit; attr(*, "non-esti

In [16]:
# Stacked Regression Ensemble
stacked_data <- data.frame(
  lm = predict(models$lm, data),
  rf = predict(models$rf, data),
  svm = predict(models$svm, data)
)
stacked_data$Actual <- data$CosineSimilarity

stacked_model <- train(Actual ~ ., data = stacked_data, method = "lm")


In [22]:
# Evaluate Scores
predictions <- predict(stacked_model, stacked_data)
data$PredictedScore <- formatC(data$PredictedScore,format = "e",digits = 3)
head(data,10)

,id,question,desired_answer,student_answer,score_me,score_other,score_avg,StudentAnswer,ModelAnswer,CosineSimilarity,PredictedScore
,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<chr>
1,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,"For us it is usually a set, get, and displayMessge or other messages that tell you what properties a class has but does not reveal its implementations. You could have anything you want in your own class definition.",1,3,2.0,us usually set get displaymessge messages tell properties class reveal implementations anything want class definition,data members attributes member functions,0.0000000,0.000e+00
2,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,The attributes of said class. Also whether or not it is a subclass. Also whether it is public private or protected.,4,4,4.0,attributes said class also whether subclass also whether public private protected,data members attributes member functions,0.1154701,1.150e-01
3,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,Information telling the compiler what data members and member functions belong to the class.,5,5,5.0,information telling compiler data members member functions belong class,data members attributes member functions,0.5962848,5.960e-01
4,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,"Since class is user defined, the body of the declaration can contain members, that can be either data or function declarations, and optionally access specifiers. Example for specifiers would be public, private, and protected. There is the keyword Class and the Class body is enclosed with brackets ({}).",5,5,5.0,since class user defined body declaration can contain members can either data function declarations optionally access specifiers example specifiers public private protected keyword class class body enclosed brackets,data members attributes member functions,0.1414214,1.410e-01
5,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,An object and data.,1,3,2.0,object data,data members attributes member functions,0.3162278,3.160e-01
6,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,"It's specific class name, includes only information defined by that class... implementation details... etc.",2,4,3.0,specific class name includes information defined class implementation details etc,data members attributes member functions,0.0000000,0.000e+00
7,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,data members and member functions.,5,5,5.0,data members member functions,data members attributes member functions,0.8944272,8.940e-01
8,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,A class definition usually contains the function and its data members,5,5,5.0,class definition usually contains function data members,data members attributes member functions,0.3380617,3.380e-01
9,2.1,What is typically included in a class definition?,Data members (attributes) and member functions.,Data members and member functions,5,5,5.0,data members member functions,data members attributes member functions,0.8944272,8.940e-01
